In [1]:
from eval_metrics import *

import torch
import pytorch_lightning as pl

from model import FlightModel

In [27]:
from omegaconf import OmegaConf

def get_experiment_dirs(experiment_name, models_relative_dir = None):
    if models_relative_dir is None:
        models_relative_dir = "models/"

    experiment_dir = models_relative_dir + experiment_name + "/"
    config_path = experiment_dir + "config/config.yaml"
    last_ckpt_path = experiment_dir + "models/last.ckpt"

    # Load the config
    # config_file_path = 'path/to/your/config.yaml'
    # cfg = OmegaConf.load(config_file_path)
    # return cfg
    return experiment_dir, config_path, last_ckpt_path


def filter_params(func, param_dict):
    sig = inspect.signature(func)
    return {k: v for k, v in param_dict.items() if k in sig.parameters}


In [14]:
from omegaconf import OmegaConf
from model import FlightModel  # Ensure this is your correct model import
import torch

def instantiate_flight_model(config_path, checkpoint_path, map_location = None):
    # Load the config
    cfg = OmegaConf.load(config_path)
    
    # Extract model configuration and remove _target_
    model_cfg = cfg['model']['model']
    model_cfg = {k: v for k, v in model_cfg.items() if k != '_target_'}
    
    # Instantiate the model
    model = FlightModel(**model_cfg)
    
    # Load the checkpoint manually
    if map_location is None:
        map_location = 'cpu'
    checkpoint = torch.load(checkpoint_path, map_location=map_location)

    # Load the state dict into the model
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    
    return model

# Path to your config file and checkpoint file
config_path = "models/TestHydra/config/config.yaml"
checkpoint_path = "models/TestHydra/models/last.ckpt"

# Instantiate the model using the config and checkpoint
flight_model = instantiate_flight_model(config_path, checkpoint_path)

# Print to confirm model instantiation
print(flight_model)


Using Default model weights initialization
FlightModel()


/var/folders/3v/j_z4lh492653rkrwnbw6_p6w0000gn/T/ipykernel_54919/701295282.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_

In [ ]:
coordinate_system_enum = CoordinateEnum.LatLongCoordinates
coordinate_system = helper_get_coordinate_system_based_on_enum(coordinate_enum=coordinate_system_enum)

auxiliary_input_channels = [
                            "diff_time", 
                            "flight_course_corrected", 
                            "flight_course_unknown", 
                            ]

auxiliary_output_channels = []

len_coordinate_system  = len(coordinate_system)
desired_input_features = coordinate_system  + auxiliary_input_channels
desired_output_features = coordinate_system  + auxiliary_output_channels
desired_features_input_and_output = coordinate_system  + auxiliary_input_channels + auxiliary_output_channels

num_input_features = len(desired_input_features)
num_output_features = len(desired_output_features)

num_res_blocks = 4
intermediate_channels = 64
in_channels = len(coordinate_system) + len(auxiliary_input_channels)
out_channels = len(coordinate_system)





num_workers = 1

num_input_rows_total = 100
min_rows_input = 100
num_output_rows = 1

individual_flights_dir = "/Users/aleksandranikevich/Desktop/AircraftTrajectory/data/Individual_Flights/"
flight_dfs = create_csv_dict(individual_flights_dir)

datamodule = Datamodule(all_flight_dataframes_dict = flight_dfs, 
                        num_input_rows_total = num_input_rows_total, 
                        min_rows_input = min_rows_input, 
                        num_output_rows = 1, 
                        coordinate_system_enum = coordinate_system_enum,
                        auxiliary_input_channels = auxiliary_input_channels,
                        auxiliary_output_channels = auxiliary_output_channels,
                        train_prop = 0.8, 
                        batch_size = 32, 
                        num_workers = num_workers, 
                        pin_memory = True,)

train_dataloader = datamodule.train_dataloader()
test_dataloader = datamodule.test_dataloader()

# dummy tensor
test_dataloader_iterator = test_dataloader.__iter__()
tensors_dict = next(test_dataloader_iterator) # dict_keys(['input_tensor', 'output_tensor', 'zero_pad_rows', 'chunk_index', 'normalization_tensor', 'meta_flightpath', 'meta_msn'])
# shape: tensors_dict["input_tensor"].shape

In [42]:
cfg = OmegaConf.load(config_path)

# Extract model configuration and remove _target_
model_cfg = cfg['model']['model'] 

In [43]:
cfg = OmegaConf.load(config_path)

# Extract model configuration and remove _target_
model_cfg = cfg['model']['model'] 

flight_model_instantiated = instantiate(model_cfg, _recursive_=True)
_ = flight_model_instantiated(example_input)

Using Default model weights initialization


In [ ]:
example_input = tensors_dict["input_tensor"]
scripted_model = torch.jit.trace(flight_model, example_input)

In [44]:
# Initialize the trainer
trainer = Trainer()

# Use the trainer to ensure the model is properly prepared
trainer.validate(flight_model_instantiated, dataloaders=[test_dataloader, test_dataloader])  #datamodule=datamodule)    # dataloaders=test_dataloader)

# Ensure model is in evaluation mode
flight_model.eval()

# Example input for tracing
example_input = tensors_dict["input_tensor"]

# Trace the model
scripted_model = torch.jit.trace(flight_model, example_input)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/trajectory/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Validation: |          | 0/? [00:00<?, ?it/s]

debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flightpath_chunk_given_to_model.shape:  (2, 100)
debug flig


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [15]:
flight_model.model

{'_target_': 'net.SimpleNet', 'in_channels': 5, 'out_channels': 2, 'intermediate_channels': 64, 'num_res_blocks': 8, 'num_output_rows': 1, 'dilation': 1, 'kernel_size': 9, 'use_bn_norm': True, 'stride': 1, 'bias': True}

In [ ]:
model = FlightModel.load_from_checkpoint(checkpoint_path=checkpoint_path, 
                                         map_location='cpu', 
                                         model=net, 
                                         loss_fn=loss_fn, 
                                         optimizer=optimizer, 
                                         max_num_val_maps=max_num_val_maps, 
                                         n_future_timesteps=10, 
                                         mean=mean, 
                                         std=std)

In [39]:
flight_model_instantiated = instantiate(model_cfg, _recursive_=True)
_ = flight_model_instantiated(example_input)

Using Default model weights initialization


In [ ]:
# Initialize the trainer
trainer = Trainer()

# Use the trainer to ensure the model is properly prepared
trainer.validate(flight_model_instantiated, dataloaders=[test_dataloader, test_dataloader])  #datamodule=datamodule)    # dataloaders=test_dataloader)

# Ensure model is in evaluation mode
flight_model.eval()

# Example input for tracing
example_input = tensors_dict["input_tensor"]

# Trace the model
scripted_model = torch.jit.trace(flight_model, example_input)

In [41]:
scripted_model = torch.jit.trace(flight_model_instantiated, example_input)

RuntimeError: FlightModel is not attached to a `Trainer`.

In [40]:
import torch
from pytorch_lightning import Trainer

# Instantiate the trainer
trainer = Trainer()

# Attach the trainer to the model
trainer.model = flight_model

# Ensure model is in evaluation mode
flight_model.eval()

# Example input for tracing
#example_input = tensors_dict["input_tensor"]

# Trace the model
scripted_model = torch.jit.trace(flight_model_instantiated, example_input)

# # Save the scripted model
# torch.jit.save(scripted_model, "path/to/save/scripted_model.pt")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


AttributeError: property 'model' of 'Trainer' object has no setter

In [25]:
# model = FlightModel.load_from_checkpoint(checkpoint_path=model_path, 
#                                          map_location='cpu', 
#                                          model=net, 
#                                          loss_fn=loss_fn, 
#                                          optimizer=optimizer, 
#                                          max_num_val_maps=max_num_val_maps, 
#                                          n_future_timesteps=10, 
#                                          mean=mean, 
#                                          std=std)



cfg = OmegaConf.load(config_path)

# Extract model configuration and remove _target_
model_cfg = cfg['model']['model'] #['model']


new_model = FlightModel.load_from_checkpoint(**model_cfg, checkpoint_path=checkpoint_path)

Using Default model weights initialization


RuntimeError: Error(s) in loading state_dict for FlightModel:
	Unexpected key(s) in state_dict: "model.first_conv_block.conv.weight", "model.first_conv_block.conv.bias", "model.first_conv_block.pw_conv.weight", "model.first_conv_block.pw_conv.bias", "model.first_conv_block.act.weight", "model.first_conv_block.bn.running_mean", "model.first_conv_block.bn.running_var", "model.first_conv_block.bn.num_batches_tracked", "model.residual_blocks.0.block1.conv.weight", "model.residual_blocks.0.block1.conv.bias", "model.residual_blocks.0.block1.pw_conv.weight", "model.residual_blocks.0.block1.pw_conv.bias", "model.residual_blocks.0.block1.act.weight", "model.residual_blocks.0.block1.bn.running_mean", "model.residual_blocks.0.block1.bn.running_var", "model.residual_blocks.0.block1.bn.num_batches_tracked", "model.residual_blocks.0.block2.conv.weight", "model.residual_blocks.0.block2.conv.bias", "model.residual_blocks.0.block2.pw_conv.weight", "model.residual_blocks.0.block2.pw_conv.bias", "model.r

In [31]:
model_cfg_inst = {key: val for key, val in model_cfg.items() if key != "model" and key != "_target_"}
model_cfg_inst

{'coordinate_system_enum': 'LatLongCoordinates',
 'loss_fn': 'mse',
 'optimizer': None,
 'max_num_val_maps': 5,
 'n_future_timesteps': 10,
 'mean': None,
 'std': None,
 'learning_rate': 0.0001}

In [34]:
from hydra.utils import instantiate
net = instantiate(model_cfg.model)


In [37]:
model_cfg_recursive = instantiate(model_cfg, _recursive_=True)
model_cfg_recursive(example_input)

Using Default model weights initialization


tensor([[[  34.9476],
         [-106.4049]],

        [[  37.0468],
         [-124.6800]],

        [[  38.4501],
         [ -76.8622]],

        [[  33.9349],
         [-118.3910]],

        [[  32.3719],
         [ -64.7853]],

        [[  35.2137],
         [ -80.9447]],

        [[  41.9781],
         [ -87.8940]],

        [[  36.3232],
         [ -93.8248]],

        [[  39.6347],
         [-104.9156]],

        [[  31.6342],
         [ -97.2371]],

        [[  41.9827],
         [ -87.9134]],

        [[  36.2650],
         [-115.0325]],

        [[  38.9425],
         [ -77.4561]],

        [[  39.8706],
         [ -86.7688]],

        [[  36.6552],
         [-119.5181]],

        [[  33.8050],
         [-115.8288]],

        [[  29.1167],
         [ -98.4808]],

        [[  35.2774],
         [ -82.4958]],

        [[  36.3459],
         [-119.9311]],

        [[  38.7390],
         [ -90.7210]],

        [[  32.1941],
         [-117.1269]],

        [[  37.4157],
         [ -

In [32]:
new_model = FlightModel.load_from_checkpoint(**model_cfg_inst, checkpoint_path=checkpoint_path)

TypeError: FlightModel.__init__() missing 1 required positional argument: 'model'

In [4]:
map_location = "cpu"
checkpoint = torch.load(checkpoint_path, map_location=map_location)
# Load the state dict into the model
flight_model.load_state_dict(checkpoint['state_dict'], strict=False)

/var/folders/3v/j_z4lh492653rkrwnbw6_p6w0000gn/T/ipykernel_54919/1025256047.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_

_IncompatibleKeys(missing_keys=[], unexpected_keys=['model.first_conv_block.conv.weight', 'model.first_conv_block.conv.bias', 'model.first_conv_block.pw_conv.weight', 'model.first_conv_block.pw_conv.bias', 'model.first_conv_block.act.weight', 'model.first_conv_block.bn.running_mean', 'model.first_conv_block.bn.running_var', 'model.first_conv_block.bn.num_batches_tracked', 'model.residual_blocks.0.block1.conv.weight', 'model.residual_blocks.0.block1.conv.bias', 'model.residual_blocks.0.block1.pw_conv.weight', 'model.residual_blocks.0.block1.pw_conv.bias', 'model.residual_blocks.0.block1.act.weight', 'model.residual_blocks.0.block1.bn.running_mean', 'model.residual_blocks.0.block1.bn.running_var', 'model.residual_blocks.0.block1.bn.num_batches_tracked', 'model.residual_blocks.0.block2.conv.weight', 'model.residual_blocks.0.block2.conv.bias', 'model.residual_blocks.0.block2.pw_conv.weight', 'model.residual_blocks.0.block2.pw_conv.bias', 'model.residual_blocks.0.block2.act.weight', 'model.

In [5]:
coordinate_system_enum = CoordinateEnum.LatLongCoordinates
coordinate_system = helper_get_coordinate_system_based_on_enum(coordinate_enum=coordinate_system_enum)

auxiliary_input_channels = [
                            "diff_time", 
                            "flight_course_corrected", 
                            "flight_course_unknown", 
                            ]

auxiliary_output_channels = []

len_coordinate_system  = len(coordinate_system)
desired_input_features = coordinate_system  + auxiliary_input_channels
desired_output_features = coordinate_system  + auxiliary_output_channels
desired_features_input_and_output = coordinate_system  + auxiliary_input_channels + auxiliary_output_channels

num_input_features = len(desired_input_features)
num_output_features = len(desired_output_features)

num_res_blocks = 4
intermediate_channels = 64
in_channels = len(coordinate_system) + len(auxiliary_input_channels)
out_channels = len(coordinate_system)





num_workers = 1

num_input_rows_total = 100
min_rows_input = 100
num_output_rows = 1

individual_flights_dir = "/Users/aleksandranikevich/Desktop/AircraftTrajectory/data/Individual_Flights/"
flight_dfs = create_csv_dict(individual_flights_dir)

datamodule = Datamodule(all_flight_dataframes_dict = flight_dfs, 
                        num_input_rows_total = num_input_rows_total, 
                        min_rows_input = min_rows_input, 
                        num_output_rows = 1, 
                        coordinate_system_enum = coordinate_system_enum,
                        auxiliary_input_channels = auxiliary_input_channels,
                        auxiliary_output_channels = auxiliary_output_channels,
                        train_prop = 0.8, 
                        batch_size = 32, 
                        num_workers = num_workers, 
                        pin_memory = True,)

train_dataloader = datamodule.train_dataloader()
test_dataloader = datamodule.test_dataloader()

# dummy tensor
test_dataloader_iterator = test_dataloader.__iter__()
tensors_dict = next(test_dataloader_iterator) # dict_keys(['input_tensor', 'output_tensor', 'zero_pad_rows', 'chunk_index', 'normalization_tensor', 'meta_flightpath', 'meta_msn'])
# shape: tensors_dict["input_tensor"].shape

In [ ]:
# example_input = torch.randn(1, input_dim)  # Example input tensor with the appropriate dimensions
# scripted_model = torch.jit.trace(flight_model, example_input)
# torch.jit.save(scripted_model, "path/to/save/scripted_model.pt")

In [7]:
# Initialize the trainer
trainer = Trainer()

# Use the trainer to ensure the model is properly prepared
trainer.validate(flight_model, dataloaders=[test_dataloader, test_dataloader])  #datamodule=datamodule)    # dataloaders=test_dataloader)

# Ensure model is in evaluation mode
flight_model.eval()

# Example input for tracing
example_input = tensors_dict["input_tensor"]

# Trace the model
scripted_model = torch.jit.trace(flight_model, example_input)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Validation: |          | 0/? [00:00<?, ?it/s]

TypeError: 'DictConfig' object is not callable

In [10]:
example_input = tensors_dict["input_tensor"]
flight_model(example_input)

TypeError: 'DictConfig' object is not callable

In [13]:
type(flight_model.model)

omegaconf.dictconfig.DictConfig

In [28]:
# import torch
# from pytorch_lightning import Trainer
# from omegaconf import OmegaConf
# from model import FlightModel
# from torch.utils.data import DataLoader, TensorDataset

# def instantiate_flight_model(config_path, checkpoint_path, map_location=None):
#     cfg = OmegaConf.load(config_path)
#     model_cfg = cfg['model']['model']
#     model_cfg = {k: v for k, v in model_cfg.items() if k != '_target_'}

#     model = FlightModel(**model_cfg)

#     if map_location is None:
#         map_location = 'cpu'
#     checkpoint = torch.load(checkpoint_path, map_location=map_location)
#     model.load_state_dict(checkpoint['state_dict'], strict=False)

#     return model

# config_path = "models/TestHydra/config/config.yaml"
# checkpoint_path = "models/TestHydra/models/last.ckpt"
# flight_model = instantiate_flight_model(config_path, checkpoint_path)
# print(flight_model)

# # Dummy validation data loader
# dummy_val_loader = DataLoader(TensorDataset(torch.randn(10, input_dim), torch.randn(10, output_dim)), batch_size=1)

# # Initialize the trainer
# trainer = Trainer()

# # Ensure the model is prepared by running validation
# trainer.validate(flight_model, val_dataloaders=dummy_val_loader)

# # Set the model to evaluation mode
# flight_model.eval()

# # Example input for tracing
# example_input = tensors_dict["input_tensor"]

# # Trace the model
# scripted_model = torch.jit.trace(flight_model, example_input)

# # Save the scripted model
# torch.jit.save(scripted_model, "path/to/save/scripted_model.pt")


NameError: name 'cfg' is not defined

In [27]:
import torch
from pytorch_lightning import Trainer

# Instantiate the trainer
trainer = Trainer()

# Attach the trainer to the model
trainer.model = flight_model

# Ensure model is in evaluation mode
flight_model.eval()

# Example input for tracing
example_input = tensors_dict["input_tensor"]

# Trace the model
scripted_model = torch.jit.trace(flight_model, example_input)

# # Save the scripted model
# torch.jit.save(scripted_model, "path/to/save/scripted_model.pt")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


AttributeError: property 'model' of 'Trainer' object has no setter

In [ ]:
# import torch
# from pytorch_lightning import Trainer

# # Instantiate the trainer
# trainer = Trainer()

# # Attach the trainer to the model
# trainer.model = flight_model

# # Ensure model is in evaluation mode
# flight_model.eval()

# # Example input for tracing
# example_input = tensors_dict["input_tensor"]

# # Trace the model
# scripted_model = torch.jit.trace(flight_model, example_input)

# # Save the scripted model
# torch.jit.save(scripted_model, "path/to/save/scripted_model.pt")


In [26]:
example_input = tensors_dict["input_tensor"]
scripted_model = torch.jit.trace(flight_model, example_input)

RuntimeError: FlightModel is not attached to a `Trainer`.

In [22]:
scripted_model = flight_model.to_torchscript()

RuntimeError: Can't redefine method: forward on class: __torch__.model.FlightModel (of Python compilation unit at: 0x12a121cf8)

In [16]:

from hydra.utils import instantiate
from omegaconf import OmegaConf
from model import FlightModel  # Ensure this is your correct model import
import torch

def instantiate_flight_model(config_path, checkpoint_path, map_location=None):
    # Load the config
    cfg = OmegaConf.load(config_path)
    
    # Extract model configuration and remove _target_
    model_cfg = cfg['model']['model']
    model_cfg = {k: v for k, v in model_cfg.items() if k != '_target_'}
    
    # Instantiate the actual model
    model_instance = instantiate(model_cfg)  # Assuming model_cfg can be instantiated to a nn.Module
    
    # Additional parameters
    other_params = {
        'coordinate_system_enum': cfg.coordinate_system.coordinate_system.coordinate_system_enum,
        'loss_fn': cfg.loss_fn,
        'optimizer': instantiate(cfg.optimizer),
        'max_num_val_maps': cfg.max_num_val_maps,
        'n_future_timesteps': cfg.n_future_timesteps,
        'mean': cfg.mean,
        'std': cfg.std,
        'learning_rate': cfg.learning_rate
    }
    
    # Instantiate the FlightModel
    flight_model = FlightModel(model=model_instance, **other_params)
    
    # Load the checkpoint manually
    if map_location is None:
        map_location = 'cpu'
    checkpoint = torch.load(checkpoint_path, map_location=map_location)
    # Load the state dict into the model
    flight_model.load_state_dict(checkpoint['state_dict'], strict=False)
    
    return flight_model

# Path to your config file and checkpoint file
config_path = "models/TestHydra/config/config.yaml"
checkpoint_path = "models/TestHydra/models/last.ckpt"

# Instantiate the model using the config and checkpoint
flight_model = instantiate_flight_model(config_path, checkpoint_path)

# Print to confirm model instantiation
print(flight_model)

# Convert

ConfigAttributeError: Missing key loss_fn
    full_key: loss_fn
    object_type=dict

In [5]:
scripted_model = flight_model.to_torchscript()

RuntimeError: 
Module 'FlightModel' has no attribute 'model' (This attribute exists on the Python module, but we failed to convert Python type: 'omegaconf.dictconfig.DictConfig' to a TorchScript type. Only tensors and (possibly nested) tuples of tensors, lists, or dictsare supported as inputs or outputs of traced functions, but instead got value of type DictConfig.. Its type was inferred; try adding a type annotation for the attribute.):
  File "/Users/aleksandranikevich/Desktop/AircraftTrajectory/REPO/flight_pattern_of_life/model.py", line 40
    def forward(self, x):
        return self.model(x)
               ~~~~~~~~~~ <--- HERE


In [7]:
flight_model.model.to_torchscript()

ConfigAttributeError: Missing key to_torchscript
    full_key: model.model.model.to_torchscript
    object_type=dict

In [8]:
type(flight_model), type(flight_model.model)

(model.FlightModel, omegaconf.dictconfig.DictConfig)

In [9]:
# example_input = torch.randn(1, input_dim)  # Example input tensor with the appropriate dimensions
# scripted_model = torch.jit.trace(flight_model, example_input)
# torch.jit.save(scripted_model, "path/to/save/scripted_model.pt")


In [26]:
from omegaconf import OmegaConf

def get_experiment_dirs(experiment_name, models_relative_dir = None):
    if models_relative_dir is None:
        models_relative_dir = "models/"

    experiment_dir = models_relative_dir + experiment_name + "/"
    config_path = experiment_dir + "config/config.yaml"
    last_ckpt_path = experiment_dir + "models/last.ckpt"

    # Load the config
    # config_file_path = 'path/to/your/config.yaml'
    # cfg = OmegaConf.load(config_file_path)
    # return cfg
    return experiment_dir, config_path, last_ckpt_path

experiment_name = "TestHydra"
experiment_dir, config_path, last_ckpt_path = get_experiment_dirs(experiment_name=experiment_name)

cfg = OmegaConf.load(config_path)

model = FlightModel.load_from_checkpoint(last_ckpt_path, **cfg['model']['model']['model'])


TypeError: FlightModel.__init__() missing 4 required positional arguments: 'model', 'coordinate_system_enum', 'loss_fn', and 'optimizer'

In [27]:
# TypeError: FlightModel.__init__() missing 4 required positional arguments: 'model', 'coordinate_system_enum', 'loss_fn', and 'optimizer'


# model:
#   _target_: model.FlightModel
#   model: ${network.network}
#   coordinate_system_enum: ${coordinate_system.coordinate_system.coordinate_system_enum}
#   loss_fn: "mse"
#   optimizer: null
#   max_num_val_maps: 5
#   n_future_timesteps: 10
#   mean: null 
#   std: null 
#   learning_rate: 1.0e-4

In [28]:
from hydra import instantiate 
FlightModel

ImportError: cannot import name 'instantiate' from 'hydra' (/opt/anaconda3/envs/trajectory/lib/python3.12/site-packages/hydra/__init__.py)

In [34]:
model_cfg = cfg['model']['model']
model = FlightModel(**model_cfg)

#and get:



TypeError: FlightModel.__init__() got an unexpected keyword argument '_target_'

In [42]:
from omegaconf import OmegaConf
from model import FlightModel  # Make sure to import your model class

model_cfg = OmegaConf.to_container(cfg['model']['model'], resolve=True)
model_cfg.pop('_target_', None)

# Instantiate the model
model = FlightModel(**model_cfg)

# from omegaconf import OmegaConf
# from model import FlightModel  # Make sure to import your model class

# def instantiate_flight_model(config_path):
#     # Load the config
#     cfg = OmegaConf.load(config_path)
    
#     # Extract model configuration and remove _target_
#     model_cfg = OmegaConf.to_container(cfg['model']['model'], resolve=True)
#     model_cfg.pop('_target_', None)
    
#     # Instantiate the model
#     model = FlightModel(**model_cfg)
    
#     return model

# # Path to your config file
# config_path = "models/TestHydra/config/config.yaml"

# # Instantiate the model using the config
# flight_model = instantiate_flight_model(config_path)

# # Print to confirm model instantiation
# print(flight_model)


Using Default model weights initialization


In [43]:
model_cfg

{'model': {'_target_': 'net.SimpleNet',
  'in_channels': 5,
  'out_channels': 2,
  'intermediate_channels': 64,
  'num_res_blocks': 8,
  'num_output_rows': 1,
  'dilation': 1,
  'kernel_size': 9,
  'use_bn_norm': True,
  'stride': 1,
  'bias': True},
 'coordinate_system_enum': 'LatLongCoordinates',
 'loss_fn': 'mse',
 'optimizer': None,
 'max_num_val_maps': 5,
 'n_future_timesteps': 10,
 'mean': None,
 'std': None,
 'learning_rate': 0.0001}

In [49]:
from omegaconf import OmegaConf
from model import FlightModel  # Ensure this is your correct model import
import torch

def instantiate_flight_model(config_path, checkpoint_path, map_location = None):
    # Load the config
    cfg = OmegaConf.load(config_path)
    
    # Extract model configuration and remove _target_
    model_cfg = cfg['model']['model']
    model_cfg = {k: v for k, v in model_cfg.items() if k != '_target_'}
    
    # Instantiate the model
    model = FlightModel(**model_cfg)
    
    # Load the checkpoint manually
    if map_location is None:
        map_location = 'cpu'
    checkpoint = torch.load(checkpoint_path, map_location=map_location)

    # Load the state dict into the model
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    
    return model

# Path to your config file and checkpoint file
config_path = "models/TestHydra/config/config.yaml"
checkpoint_path = "models/TestHydra/models/last.ckpt"

# Instantiate the model using the config and checkpoint
flight_model = instantiate_flight_model(config_path, checkpoint_path)

# Print to confirm model instantiation
print(flight_model)


Using Default model weights initialization
FlightModel()


/var/folders/3v/j_z4lh492653rkrwnbw6_p6w0000gn/T/ipykernel_49828/701295282.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_

In [50]:
# Convert to TorchScript
scripted_model = flight_model.to_torchscript()


RuntimeError: 
Module 'FlightModel' has no attribute 'model' (This attribute exists on the Python module, but we failed to convert Python type: 'omegaconf.dictconfig.DictConfig' to a TorchScript type. Only tensors and (possibly nested) tuples of tensors, lists, or dictsare supported as inputs or outputs of traced functions, but instead got value of type DictConfig.. Its type was inferred; try adding a type annotation for the attribute.):
  File "/Users/aleksandranikevich/Desktop/AircraftTrajectory/REPO/flight_pattern_of_life/model.py", line 40
    def forward(self, x):
        return self.model(x)
               ~~~~~~~~~~ <--- HERE


In [ ]:
torch.jit.save(scripted_model, "path/to/save/model.pt")

In [48]:
jit_model_path = "/Users/aleksandranikevich/Desktop/AircraftTrajectory/REPO/flight_pattern_of_life/models/TestHydra/model_jit/scripted_model.pt"

loaded_model = torch.jit.load(jit_model_path)

In [52]:
example_input.shape

torch.Size([32, 5, 100])

In [53]:
torch.rand(32, 5, 100).shape

torch.Size([32, 5, 100])

In [51]:
out_loaded = loaded_model(example_input)
out_loaded.detach().cpu().numpy().tolist()

[[[34.98129653930664], [-106.4134521484375]],
 [[37.031436920166016], [-125.01054382324219]],
 [[38.54348373413086], [-76.85899353027344]],
 [[33.9350471496582], [-118.39100646972656]],
 [[33.264122009277344], [-63.52388381958008]],
 [[35.21385955810547], [-80.94493865966797]],
 [[41.981910705566406], [-87.88936614990234]],
 [[36.39521408081055], [-93.5274887084961]],
 [[39.696327209472656], [-104.80913543701172]],
 [[31.626739501953125], [-97.22686767578125]],
 [[41.98382568359375], [-87.91226959228516]],
 [[36.27254867553711], [-115.00944519042969]],
 [[38.94214630126953], [-77.45616912841797]],
 [[39.88007736206055], [-86.7643814086914]],
 [[36.89549255371094], [-119.48739624023438]],
 [[34.634979248046875], [-113.23078918457031]],
 [[29.133026123046875], [-98.4809341430664]],
 [[35.29546356201172], [-82.49080657958984]],
 [[36.37318801879883], [-119.94331359863281]],
 [[38.741302490234375], [-90.69365692138672]],
 [[32.194087982177734], [-116.78949737548828]],
 [[38.039886474609375

In [46]:
model_cfg = cfg['model']['model']
model_cfg = {k: v for k, v in model_cfg.items() if k != '_target_'}


In [47]:
# Model Initialization
flight_model = FlightModel(**model_cfg)

# Load the Checkpoint State Dict
checkpoint = torch.load(last_ckpt_path, map_location='cpu')

# Load the State Dict into the Model
flight_model.load_state_dict(checkpoint['state_dict'], strict=False)


Using Default model weights initialization


/var/folders/3v/j_z4lh492653rkrwnbw6_p6w0000gn/T/ipykernel_49828/3852235227.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(last_ckpt_path, map_l

_IncompatibleKeys(missing_keys=[], unexpected_keys=['model.first_conv_block.conv.weight', 'model.first_conv_block.conv.bias', 'model.first_conv_block.pw_conv.weight', 'model.first_conv_block.pw_conv.bias', 'model.first_conv_block.act.weight', 'model.first_conv_block.bn.running_mean', 'model.first_conv_block.bn.running_var', 'model.first_conv_block.bn.num_batches_tracked', 'model.residual_blocks.0.block1.conv.weight', 'model.residual_blocks.0.block1.conv.bias', 'model.residual_blocks.0.block1.pw_conv.weight', 'model.residual_blocks.0.block1.pw_conv.bias', 'model.residual_blocks.0.block1.act.weight', 'model.residual_blocks.0.block1.bn.running_mean', 'model.residual_blocks.0.block1.bn.running_var', 'model.residual_blocks.0.block1.bn.num_batches_tracked', 'model.residual_blocks.0.block2.conv.weight', 'model.residual_blocks.0.block2.conv.bias', 'model.residual_blocks.0.block2.pw_conv.weight', 'model.residual_blocks.0.block2.pw_conv.bias', 'model.residual_blocks.0.block2.act.weight', 'model.

In [45]:
FlightModel.load_from_checkpoint(last_ckpt_path, **model_cfg['model'])

TypeError: FlightModel.__init__() missing 4 required positional arguments: 'model', 'coordinate_system_enum', 'loss_fn', and 'optimizer'

In [44]:
FlightModel.load_from_checkpoint(last_ckpt_path, **model_cfg)

# I get

RuntimeError                              Traceback (most recent call last)
Cell In[44], line 1
----> 1 FlightModel.load_from_checkpoint(last_ckpt_path, **model_cfg)

File /opt/anaconda3/envs/trajectory/lib/python3.12/site-packages/pytorch_lightning/utilities/model_helpers.py:125, in _restricted_classmethod_impl.__get__.<locals>.wrapper(*args, **kwargs)
    120 if instance is not None and not is_scripting:
    121     raise TypeError(
    122         f"The classmethod `{cls.__name__}.{self.method.__name__}` cannot be called on an instance."
    123         " Please call it on the class type and make sure the return value is used."
    124     )
--> 125 return self.method(cls, *args, **kwargs)

File /opt/anaconda3/envs/trajectory/lib/python3.12/site-packages/pytorch_lightning/core/module.py:1582, in LightningModule.load_from_checkpoint(cls, checkpoint_path, map_location, hparams_file, strict, **kwargs)
   1493 @_restricted_classmethod
   1494 def load_from_checkpoint(
   1495     cls,
   (...)
   1500     **kwargs: Any,
   1501 ) -> Self:
   1502     r"""Primary way of loading a model from a checkpoint. When Lightning saves a checkpoint it stores the arguments
   1503     passed to ``__init__``  in the checkpoint under ``"hyper_parameters"``.
   1504 
   (...)
   1580 
...
   2216                        self.__class__.__name__, "\n\t".join(error_msgs)))
   2217 return _IncompatibleKeys(missing_keys, unexpected_keys)

RuntimeError: Error(s) in loading state_dict for FlightModel:
	Unexpected key(s) in state_dict: "model.first_conv_block.conv.weight", "model.first_conv_block.conv.bias", "model.first_conv_block.pw_conv.weight", "model.first_conv_block.pw_conv.bias", "model.first_conv_block.act.weight", "model.first_conv_block.bn.running_mean", "model.first_conv_block.bn.running_var", "model.first_conv_block.bn.num_batches_tracked", "model.residual_blocks.0.block1.conv.weight", "model.residual_blocks.0.block1.conv.bias", "model.residual_blocks.0.block1.pw_conv.weight", "model.residual_blocks.0.block1.pw_conv.bias", "model.residual_blocks.0.block1.act.weight", "model.residual_blocks.0.block1.bn.running_mean", "model.residual_blocks.0.block1.bn.running_var", "model.residual_blocks.0.block1.bn.num_batches_tracked", "model.residual_blocks.0.block2.conv.weight", "model.residual_blocks.0.block2.conv.bias", "model.residual_blocks.0.block2.pw_conv.weight", "model.residual_blocks.0.block2.pw_conv.bias", "model.residual_blocks.0.block2.act.weight", "model.residual_blocks.0.block2.bn.running_mean", "model.residual_blocks.0.block2.bn.running_var", "model.residual_blocks.0.block2.bn.num_batches_tracked", "model.residual_blocks.1.block1.conv.weight", "model.residual_blocks.1.block1.conv.bias", "model.residual_blocks.1.block1.pw_conv.weight", "model.residual_blocks.1.block1.pw_conv.bias", "model.residual_blocks.1.block1.act.weight", "model.residual_blocks.1.block1.bn.running_mean", "model.residual_blocks.1.block1.bn.running_var", "model.residual_blocks.1.block1.bn.num_batches_tracked", "model.residual_blocks.1.block2.conv.weight", "model.residual_blocks.1.block2.conv.bias", "model.residual_blocks.1.block2.pw_conv.weight", "model.residual_blocks.1.block2.pw_conv.bias", "model.residual_blocks.1.block2.act.weight", "model.residual_blocks.1.block2.bn.running_mean", "model.residual_blocks.1.block2.bn.running_var", "model.residual_blocks.1.block2.bn.num_batches_tracked", "model.residual_blocks.2.block1.conv.weight", "model.residual_blocks.2.block1.conv.bias", "model.residual_blocks.2.block1.pw_conv.weight", "model.residual_blocks.2.block1.pw_conv.bias", "model.residual_blocks.2.block1.act.weight", "model.residual_blocks.2.block1.bn.running_mean", "model.residual_blocks.2.block1.bn.running_var", "model.residual_blocks.2.block1.bn.num_batches_tracked", "model.residual_blocks.2.block2.conv.weight", "model.residual_blocks.2.block2.conv.bias", "model.residual_blocks.2.block2.pw_conv.weight", "model.residual_blocks.2.block2.pw_conv.bias", "model.residual_blocks.2.block2.act.weight", "model.residual_blocks.2.block2.bn.running_mean", "model.residual_blocks.2.block2.bn.running_var", "model.residual_blocks.2.block2.bn.num_batches_tracked", "model.residual_blocks.3.block1.conv.weight", "model.residual_blocks.3.block1.conv.bias", "model.residual_blocks.3.block1.pw_conv.weight", "model.residual_blocks.3.block1.pw_conv.bias", "model.residual_blocks.3.block1.act.weight", "model.residual_blocks.3.block1.bn.running_mean", "model.residual_blocks.3.block1.bn.running_var", "model.residual_blocks.3.block1.bn.num_batches_tracked", "model.residual_blocks.3.block2.conv.weight", "model.residual_blocks.3.block2.conv.bias", "model.residual_blocks.3.block2.pw_conv.weight", "model.residual_blocks.3.block2.pw_conv.bias", "model.residual_blocks.3.block2.act.weight", "model.residual_blocks.3.block2.bn.running_mean", "model.residual_blocks.3.block2.bn.running_var", "model.residual_blocks.3.block2.bn.num_batches_tracked", "model.residual_blocks.4.block1.conv.weight", "model.residual_blocks.4.block1.conv.bias", "model.residual_blocks.4.block1.pw_conv.weight", "model.residual_blocks.4.block1.pw_conv.bias", "model.residual_blocks.4.block1.act.weight", "model.residual_blocks.4.block1.bn.running_mean", "model.residual_blocks.4.block1.bn.running_var", "model.residual_blocks.4.block1.bn.num_batches_tracked", "model.residual_blocks.4.block2.conv.weight", "model.residual_blocks.4.block2.conv.bias", "model.residual_blocks.4.block2.pw_conv.weight", "model.residual_blocks.4.block2.pw_conv.bias", "model.residual_blocks.4.block2.act.weight", "model.residual_blocks.4.block2.bn.running_mean", "model.residual_blocks.4.block2.bn.running_var", "model.residual_blocks.4.block2.bn.num_batches_tracked", "model.residual_blocks.5.block1.conv.weight", "model.residual_blocks.5.block1.conv.bias", "model.residual_blocks.5.block1.pw_conv.weight", "model.residual_blocks.5.block1.pw_conv.bias", "model.residual_blocks.5.block1.act.weight", "model.residual_blocks.5.block1.bn.running_mean", "model.residual_blocks.5.block1.bn.running_var", "model.residual_blocks.5.block1.bn.num_batches_tracked", "model.residual_blocks.5.block2.conv.weight", "model.residual_blocks.5.block2.conv.bias", "model.residual_blocks.5.block2.pw_conv.weight", "model.residual_blocks.5.block2.pw_conv.bias", "model.residual_blocks.5.block2.act.weight", "model.residual_blocks.5.block2.bn.running_mean", "model.residual_blocks.5.block2.bn.running_var", "model.residual_blocks.5.block2.bn.num_batches_tracked", "model.residual_blocks.6.block1.conv.weight", "model.residual_blocks.6.block1.conv.bias", "model.residual_blocks.6.block1.pw_conv.weight", "model.residual_blocks.6.block1.pw_conv.bias", "model.residual_blocks.6.block1.act.weight", "model.residual_blocks.6.block1.bn.running_mean", "model.residual_blocks.6.block1.bn.running_var", "model.residual_blocks.6.block1.bn.num_batches_tracked", "model.residual_blocks.6.block2.conv.weight", "model.residual_blocks.6.block2.conv.bias", "model.residual_blocks.6.block2.pw_conv.weight", "model.residual_blocks.6.block2.pw_conv.bias", "model.residual_blocks.6.block2.act.weight", "model.residual_blocks.6.block2.bn.running_mean", "model.residual_blocks.6.block2.bn.running_var", "model.residual_blocks.6.block2.bn.num_batches_tracked", "model.residual_blocks.7.block1.conv.weight", "model.residual_blocks.7.block1.conv.bias", "model.residual_blocks.7.block1.pw_conv.weight", "model.residual_blocks.7.block1.pw_conv.bias", "model.residual_blocks.7.block1.act.weight", "model.residual_blocks.7.block1.bn.running_mean", "model.residual_blocks.7.block1.bn.running_var", "model.residual_blocks.7.block1.bn.num_batches_tracked", "model.residual_blocks.7.block2.conv.weight", "model.residual_blocks.7.block2.conv.bias", "model.residual_blocks.7.block2.pw_conv.weight", "model.residual_blocks.7.block2.pw_conv.bias", "model.residual_blocks.7.block2.act.weight", "model.residual_blocks.7.block2.bn.running_mean", "model.residual_blocks.7.block2.bn.running_var", "model.residual_blocks.7.block2.bn.num_batches_tracked", "model.last_conv_block.conv.weight", "model.last_conv_block.conv.bias", "model.last_conv_block.pw_conv.weight", "model.last_conv_block.pw_conv.bias", "model.last_conv_block.act.weight", "model.last_conv_block.bn.running_mean", "model.last_conv_block.bn.running_var", "model.last_conv_block.bn.num_batches_tracked", "model.one_more_linear.weight", "model.one_more_linear.bias", "model.act_one_more_linear.weight", "model.last_linear.weight", "model.last_linear.bias", "model.act_final.weight". 
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

Using Default model weights initialization


RuntimeError: Error(s) in loading state_dict for FlightModel:
	Unexpected key(s) in state_dict: "model.first_conv_block.conv.weight", "model.first_conv_block.conv.bias", "model.first_conv_block.pw_conv.weight", "model.first_conv_block.pw_conv.bias", "model.first_conv_block.act.weight", "model.first_conv_block.bn.running_mean", "model.first_conv_block.bn.running_var", "model.first_conv_block.bn.num_batches_tracked", "model.residual_blocks.0.block1.conv.weight", "model.residual_blocks.0.block1.conv.bias", "model.residual_blocks.0.block1.pw_conv.weight", "model.residual_blocks.0.block1.pw_conv.bias", "model.residual_blocks.0.block1.act.weight", "model.residual_blocks.0.block1.bn.running_mean", "model.residual_blocks.0.block1.bn.running_var", "model.residual_blocks.0.block1.bn.num_batches_tracked", "model.residual_blocks.0.block2.conv.weight", "model.residual_blocks.0.block2.conv.bias", "model.residual_blocks.0.block2.pw_conv.weight", "model.residual_blocks.0.block2.pw_conv.bias", "model.r

In [37]:
model.load_from_checkpoint(last_ckpt_path)

TypeError: The classmethod `FlightModel.load_from_checkpoint` cannot be called on an instance. Please call it on the class type and make sure the return value is used.

In [38]:
model_cfg

{'model': {'_target_': 'net.SimpleNet',
  'in_channels': 5,
  'out_channels': 2,
  'intermediate_channels': 64,
  'num_res_blocks': 8,
  'num_output_rows': 1,
  'dilation': 1,
  'kernel_size': 9,
  'use_bn_norm': True,
  'stride': 1,
  'bias': True},
 'coordinate_system_enum': 'LatLongCoordinates',
 'loss_fn': 'mse',
 'optimizer': None,
 'max_num_val_maps': 5,
 'n_future_timesteps': 10,
 'mean': None,
 'std': None,
 'learning_rate': 0.0001}

In [36]:
FlightModel.load_from_checkpoint(last_ckpt_path, **model_cfg)

Using Default model weights initialization


RuntimeError: Error(s) in loading state_dict for FlightModel:
	Unexpected key(s) in state_dict: "model.first_conv_block.conv.weight", "model.first_conv_block.conv.bias", "model.first_conv_block.pw_conv.weight", "model.first_conv_block.pw_conv.bias", "model.first_conv_block.act.weight", "model.first_conv_block.bn.running_mean", "model.first_conv_block.bn.running_var", "model.first_conv_block.bn.num_batches_tracked", "model.residual_blocks.0.block1.conv.weight", "model.residual_blocks.0.block1.conv.bias", "model.residual_blocks.0.block1.pw_conv.weight", "model.residual_blocks.0.block1.pw_conv.bias", "model.residual_blocks.0.block1.act.weight", "model.residual_blocks.0.block1.bn.running_mean", "model.residual_blocks.0.block1.bn.running_var", "model.residual_blocks.0.block1.bn.num_batches_tracked", "model.residual_blocks.0.block2.conv.weight", "model.residual_blocks.0.block2.conv.bias", "model.residual_blocks.0.block2.pw_conv.weight", "model.residual_blocks.0.block2.pw_conv.bias", "model.r

In [23]:
import torch

# Load the checkpoint
checkpoint = torch.load(last_ckpt_path)

# Get the state_dict from the checkpoint
checkpoint_state_dict = checkpoint['state_dict']

# # Get the state_dict from the model
# model_state_dict = model.state_dict()

# # Print keys in checkpoint
# print("Checkpoint State Dict Keys:", checkpoint_state_dict.keys())

# # Print keys in current model's state dict
# print("Model State Dict Keys:", model_state_dict.keys())


checkpoint_state_dict.keys()


/var/folders/3v/j_z4lh492653rkrwnbw6_p6w0000gn/T/ipykernel_49828/2239485421.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(last_ckpt_path)


odict_keys(['model.first_conv_block.conv.weight', 'model.first_conv_block.conv.bias', 'model.first_conv_block.pw_conv.weight', 'model.first_conv_block.pw_conv.bias', 'model.first_conv_block.act.weight', 'model.first_conv_block.bn.running_mean', 'model.first_conv_block.bn.running_var', 'model.first_conv_block.bn.num_batches_tracked', 'model.residual_blocks.0.block1.conv.weight', 'model.residual_blocks.0.block1.conv.bias', 'model.residual_blocks.0.block1.pw_conv.weight', 'model.residual_blocks.0.block1.pw_conv.bias', 'model.residual_blocks.0.block1.act.weight', 'model.residual_blocks.0.block1.bn.running_mean', 'model.residual_blocks.0.block1.bn.running_var', 'model.residual_blocks.0.block1.bn.num_batches_tracked', 'model.residual_blocks.0.block2.conv.weight', 'model.residual_blocks.0.block2.conv.bias', 'model.residual_blocks.0.block2.pw_conv.weight', 'model.residual_blocks.0.block2.pw_conv.bias', 'model.residual_blocks.0.block2.act.weight', 'model.residual_blocks.0.block2.bn.running_mean

In [25]:
cfg['model']['model']['model']

{'_target_': 'net.SimpleNet', 'in_channels': 5, 'out_channels': 2, 'intermediate_channels': 64, 'num_res_blocks': 8, 'num_output_rows': 1, 'dilation': 1, 'kernel_size': 9, 'use_bn_norm': True, 'stride': 1, 'bias': True}

In [20]:
model = FlightModel.load_from_checkpoint(last_ckpt_path, **cfg['model']['model'])



Using Default model weights initialization


RuntimeError: Error(s) in loading state_dict for FlightModel:
	Unexpected key(s) in state_dict: "model.first_conv_block.conv.weight", "model.first_conv_block.conv.bias", "model.first_conv_block.pw_conv.weight", "model.first_conv_block.pw_conv.bias", "model.first_conv_block.act.weight", "model.first_conv_block.bn.running_mean", "model.first_conv_block.bn.running_var", "model.first_conv_block.bn.num_batches_tracked", "model.residual_blocks.0.block1.conv.weight", "model.residual_blocks.0.block1.conv.bias", "model.residual_blocks.0.block1.pw_conv.weight", "model.residual_blocks.0.block1.pw_conv.bias", "model.residual_blocks.0.block1.act.weight", "model.residual_blocks.0.block1.bn.running_mean", "model.residual_blocks.0.block1.bn.running_var", "model.residual_blocks.0.block1.bn.num_batches_tracked", "model.residual_blocks.0.block2.conv.weight", "model.residual_blocks.0.block2.conv.bias", "model.residual_blocks.0.block2.pw_conv.weight", "model.residual_blocks.0.block2.pw_conv.bias", "model.r

In [ ]:
# model:
#   _target_: model.FlightModel
#   model: ${network.network}
#   coordinate_system_enum: ${coordinate_system.coordinate_system.coordinate_system_enum}
#   loss_fn: "mse"
#   optimizer: null
#   max_num_val_maps: 5
#   n_future_timesteps: 10
#   mean: null 
#   std: null 
#   learning_rate: 1.0e-4

In [16]:
os.listdir('/Users/aleksandranikevich/Desktop/AircraftTrajectory/REPO/flight_pattern_of_life/models/TestHydra/config/')

['config.yaml']

In [18]:
load_model = FlightModel.load_from_checkpoint(model_dir)

TypeError: FlightModel.__init__() missing 4 required positional arguments: 'model', 'coordinate_system_enum', 'loss_fn', and 'optimizer'

In [8]:
all_models_folder = "/Users/aleksandranikevich/Desktop/AircraftTrajectory/flight_pattern_of_life-main/models/"
experimet = "test61"
model_folder = all_models_folder + experimet + "/models/"
model_dir = model_folder + "last.ckpt"
os.listdir(model_folder)

['last.ckpt', 'epoch=0-step=5000.ckpt']

In [10]:
import os
os.getcwd()

'/Users/aleksandranikevich/Desktop/AircraftTrajectory/REPO/flight_pattern_of_life'

In [11]:
os.listdir("models/")

['TestHydra']

In [9]:
load_model = FlightModel.load_from_checkpoint(model_dir)

TypeError: FlightModel.__init__() missing 4 required positional arguments: 'model', 'coordinate_system_enum', 'loss_fn', and 'optimizer'

In [1]:
# import torch
# import pytorch_lightning as pl

# class MyModel(pl.LightningModule):
#     # Your model implementation

#     def forward(self, x):
#         # Forward pass
#         pass

# # Load your trained model
# model = MyModel.load_from_checkpoint("path/to/your/checkpoint.ckpt")

# # Convert to TorchScript
# scripted_model = model.to_torchscript()
# torch.jit.save(scripted_model, "path/to/save/model.pt")


In [11]:
from pytorch_lightning import LightningModule
class SimpleModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(in_features=64, out_features=4)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))


# create the model
model = SimpleModel()
script = model.to_torchscript()

# save for use in production environment
torch.jit.save(script, "model.pt")

In [55]:
import torch
import requests
example_input = torch.rand(3, 5, 100)

#cluster_ip = "aircraft-trajectory.demo.svc.cluster.local"
cluster_ip = "aircraft-trajectory.demo.kurrent.dev.platform.raftlabs.tech"
model_name = "aircraft-trajectory"
predict_url = f"http://{cluster_ip}/v1/models/{model_name}:predict"

# KServe expects a specific payload format
payload = {"instances": example_input.tolist()}

headers = {"Content-Type": "application/json"}

response = requests.post(predict_url,
              headers=headers,
              json=payload)

response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)